In [ ]:
import generation as gen
import utils
import files
import os
import numpy as np
import pandas as pd
from tqdm.autonotebook import tqdm
import datasets
import copy

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
from matplotlib.backends.backend_pgf import FigureCanvasPgf
mpl.backend_bases.register_backend('pdf', FigureCanvasPgf)
import matplotlib.lines as mlines

In [ ]:
size=19
mpl.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'font.serif': 'Times',
    'text.usetex': True,
    'pgf.rcfonts': False,
    'font.size': size,
    'axes.labelsize':size,
    'axes.titlesize':size,
    'figure.titlesize':size,
    'xtick.labelsize':size,
    'ytick.labelsize':size,
    'legend.fontsize':size,
})

In [ ]:
datasets_names = ['meps_19', 'bio', 'concrete', 'bike']
dataset_base_path = "./data/cqr_datasets/"

In [ ]:
datasets_sizes_normal = {'meps_19': {'train': 1000, 'cal': 500, 'test_pattern': 100},
                         'meps_20': {'train': 1000, 'cal': 500, 'test_pattern': 100},
                         'meps_21': {'train': 1000, 'cal': 500, 'test_pattern': 100},
                         'bio': {'train': 1000, 'cal': 500, 'test_pattern': 100},
                         'concrete': {'train': 630, 'cal': 200, 'test_pattern': 100},
                         'bike': {'train': 1000, 'cal': 500, 'test_pattern': 100}}

datasets_sizes_small = {'meps_19': {'train': 500, 'cal': 250, 'test_pattern': 100},
                        'meps_20': {'train': 500, 'cal': 250, 'test_pattern': 100},
                        'meps_21': {'train': 500, 'cal': 250, 'test_pattern': 100},
                        'bio': {'train': 500, 'cal': 250, 'test_pattern': 100},
                        'concrete': {'train': 330, 'cal': 100, 'test_pattern': 100},
                        'bike': {'train': 500, 'cal': 250, 'test_pattern': 100}}

In [ ]:
n_rep = 100
alpha = 0.1

In [ ]:
prob_missing = 0.2

In [ ]:
imputation = 'iterative_ridge'
mask = 'Yes'
protection = 'No'

In [ ]:
methods = ['CQR', 'CQR_MDA']#QR_TrainCal
basemodel = 'NNet'
exacts = [False, True]

In [ ]:
name_pipelines = []
for method in methods: 
    if method == 'CQR_MDA':
        name_temp = files.get_name_method(method, basemodel, mask=mask, protection=protection, exact=True)
        if not name_temp in name_pipelines:
            name_pipelines.append(name_temp)
    name_temp = files.get_name_method(method, basemodel, mask=mask, protection=protection, exact=False)
    if not name_temp in name_pipelines:
        name_pipelines.append(name_temp)
    
current_pipeline = method+'_'+basemodel

In [ ]:
if mask == 'Yes':
    dict_methods = {'QR_TrainCal_NNet_Mask': 'QR',
                    'CQR_NNet_Mask': 'CQR', 
                    'CQR_MDA_Exact_NNet_Mask': 'CQR-MDA-Exact',
                    'CQR_MDA_Nested_NNet_Mask': 'CQR-MDA-Nested'}
else:
    dict_methods = {'QR_TrainCal_NNet': 'QR',
                    'CQR_NNet': 'CQR', 
                    'CQR_MDA_Exact_NNet': 'CQR-MDA-Exact',
                    'CQR_MDA_Nested_NNet': 'CQR-MDA-Nested'}

In [ ]:
#sizes = ['small', 'normal']
sizes = ['normal']
        
dict_cov = dict.fromkeys(datasets_names)
dict_len = dict.fromkeys(datasets_names)
dict_cov_patterns = dict.fromkeys(datasets_names)
dict_len_patterns = dict.fromkeys(datasets_names)
dict_cov_worst = dict.fromkeys(datasets_names)
dict_len_worst = dict.fromkeys(datasets_names)
dict_cov_best = dict.fromkeys(datasets_names)
dict_len_best = dict.fromkeys(datasets_names)

for dataset_name in datasets_names:

    dict_cov[dataset_name] = dict.fromkeys(sizes)
    dict_len[dataset_name] = dict.fromkeys(sizes)
    dict_cov_patterns[dataset_name] = dict.fromkeys(sizes)
    dict_len_patterns[dataset_name] = dict.fromkeys(sizes)
    dict_cov_worst[dataset_name] = dict.fromkeys(sizes)
    dict_len_worst[dataset_name] = dict.fromkeys(sizes)
    dict_cov_best[dataset_name] = dict.fromkeys(sizes)
    dict_len_best[dataset_name] = dict.fromkeys(sizes)

    for size in sizes:

        dict_cov[dataset_name][size] = dict.fromkeys(name_pipelines)
        dict_len[dataset_name][size] = dict.fromkeys(name_pipelines)
        dict_cov_patterns[dataset_name][size] = dict.fromkeys(name_pipelines)
        dict_len_patterns[dataset_name][size] = dict.fromkeys(name_pipelines)
        dict_cov_worst[dataset_name][size] = dict.fromkeys(name_pipelines)
        dict_len_worst[dataset_name][size] = dict.fromkeys(name_pipelines)
        dict_cov_best[dataset_name][size] = dict.fromkeys(name_pipelines)
        dict_len_best[dataset_name][size] = dict.fromkeys(name_pipelines)

        for pipeline in name_pipelines:
            dict_cov_patterns[dataset_name][size][pipeline] = {}
            dict_len_patterns[dataset_name][size][pipeline] = {}

In [ ]:
for dataset_name in datasets_names:
    
    print(dataset_name)
    
    df, target, var_missing = datasets.GetDataset(dataset_name, dataset_base_path)

    d = df.shape[1]-1

    params_missing = {}
    params_missing['var_missing'] = var_missing
    params_missing['prob_missing'] = prob_missing

    for size in sizes:
        
        if size == 'normal':
            train_size = datasets_sizes_normal[dataset_name]['train']
            cal_size = datasets_sizes_normal[dataset_name]['cal']
            
            if dataset_name == 'concrete':
                nb_sample_pattern = datasets_sizes_normal[dataset_name]['test_pattern']
                params_test = {'iid':{'test_size': 200}, 'fixed_nb_sample_pattern':{'nb_sample_pattern': nb_sample_pattern}}
            else:
                nb_sample_pattern = datasets_sizes_normal[dataset_name]['test_pattern']
                params_test = {'iid':{'test_size': 2000}, 'fixed_nb_sample_pattern':{'nb_sample_pattern': nb_sample_pattern}}
            params_test = gen.process_test(params_test, d=d, params_missing=params_missing)

            max_test_size = np.max(params_test['test_size'])
            
        elif size == 'small':
            train_size = datasets_sizes_small[dataset_name]['train']
            cal_size = datasets_sizes_small[dataset_name]['cal']
            
            if dataset_name == 'concrete':
                nb_sample_pattern = datasets_sizes_small[dataset_name]['test_pattern']
                params_test = {'iid':{'test_size': 200}, 'fixed_nb_sample_pattern':{'nb_sample_pattern': nb_sample_pattern}}
            else:
                nb_sample_pattern = datasets_sizes_small[dataset_name]['test_pattern']
                params_test = {'iid':{'test_size': 2000}, 'fixed_nb_sample_pattern':{'nb_sample_pattern': nb_sample_pattern}}
            params_test = gen.process_test(params_test, d=d, params_missing=params_missing)

            max_test_size = np.max(params_test['test_size'])

        name_method = []

        for pipeline in name_pipelines:

            name_method = np.append(name_method, '_'.join([imputation, pipeline]))

            key = -1

            data, results = utils.get_data_results(pipeline, train_size, cal_size, params_test, n_rep, imputation=imputation,
                                                   dataset = dataset_name,
                                                   params_missing=params_missing,
                                                   parent_results='results', parent_data='data', extension='xz')

            contains, lengths = utils.compute_PI_metrics(data, results, 'iid')

            metrics = utils.compute_metrics_cond(n_rep, data, results, 'fixed_nb_sample_pattern', cond='Pattern')

            dict_cov[dataset_name][size][pipeline] = np.mean(contains, axis=1)
            dict_len[dataset_name][size][pipeline] = np.mean(lengths, axis=1)

            for key_pattern in list(metrics.keys()):

                dict_cov_patterns[dataset_name][size][pipeline][key_pattern] = metrics[key_pattern]['avg_cov']
                dict_len_patterns[dataset_name][size][pipeline][key_pattern] = metrics[key_pattern]['avg_len']

In [ ]:
for dataset_name in datasets_names:
    
    for size in sizes:
        
        for pipeline in name_pipelines:

            avg_cov = dict.fromkeys(dict_cov_patterns[dataset_name][size][pipeline].keys())

            for key in list(avg_cov.keys()):
                avg_cov[key] = np.mean(dict_cov_patterns[dataset_name][size][pipeline][key])

            worst_index = np.argmin(list(avg_cov.values()))
            worst_key = list(avg_cov.keys())[worst_index]
            dict_cov_worst[dataset_name][size][pipeline] = dict_cov_patterns[dataset_name][size][pipeline][worst_key]
            dict_len_worst[dataset_name][size][pipeline] = dict_len_patterns[dataset_name][size][pipeline][worst_key]

            best_index = np.argmax(list(avg_cov.values()))
            best_key = list(avg_cov.keys())[best_index]
            dict_cov_best[dataset_name][size][pipeline] = dict_cov_patterns[dataset_name][size][pipeline][best_key]
            dict_len_best[dataset_name][size][pipeline] = dict_len_patterns[dataset_name][size][pipeline][best_key]  

In [ ]:
colors_blindness = sns.color_palette("colorblind")
colors_blindness

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(15,4.3))
axes = {'meps_19': ax1, 'bio': ax2, 'concrete': ax3, 'bike': ax4}

dict_markers = {'iid': 'd',
                'worst': "v", 
                'best': "^"}

dict_datasets = {'meps_19': r'\texttt{meps_19} ($d=139$, $l=5$)','meps_20': r'\texttt{meps} ($d=139$, $l=5$)',
                 'meps_21': r'\texttt{meps} ($d=139$, $l=5$)','bio': r'\texttt{bio} ($d=9$, $l=9$)',
                 'concrete': r'\texttt{concrete} ($d=8$, $l=8$)','bike': r'\texttt{bike} ($d=18$, $l=4$)'}
alphas_meps = {'meps_19': 1, 'meps_20':0.7, 'meps_21': 0.4}
dict_colors = {'QR_TrainCal_NNet_Mask': colors_blindness[2],
               'CQR_NNet_Mask': colors_blindness[1], 
               'CQR_MDA_Exact_NNet_Mask': colors_blindness[4],
               'CQR_MDA_Nested_NNet_Mask': colors_blindness[9]}

dict_coverages = {'iid': 'Marginal', 'worst': 'Lowest', 'best': 'Highest'}

marker_size = 60

small = False
medium = True

name_pipelines_to_plot = name_pipelines
    
for dataset_name in datasets_names:
    
    ax = axes[dataset_name]
    ax.set_title(dict_datasets[dataset_name])

    alpha_data = 1

    for pipeline in name_pipelines_to_plot:

        if medium:
            ax.scatter(np.mean(dict_cov[dataset_name]['normal'][pipeline]),
                       np.mean(dict_len[dataset_name]['normal'][pipeline]), 
                       marker=dict_markers['iid'], color=dict_colors[pipeline],s=marker_size,alpha=alpha_data)
            ax.errorbar(np.mean(dict_cov[dataset_name]['normal'][pipeline]), 
                        np.mean(dict_len[dataset_name]['normal'][pipeline]),
                        xerr=np.std(dict_cov[dataset_name]['normal'][pipeline])/np.sqrt(n_rep),
                        yerr=np.std(dict_len[dataset_name]['normal'][pipeline])/np.sqrt(n_rep), 
                        color=dict_colors[pipeline], alpha=0.3)
            ax.scatter(np.mean(dict_cov_worst[dataset_name]['normal'][pipeline]),
                       np.mean(dict_len_worst[dataset_name]['normal'][pipeline]), 
                       marker=dict_markers['worst'], color=dict_colors[pipeline],s=marker_size,alpha=alpha_data)
            ax.errorbar(np.mean(dict_cov_worst[dataset_name]['normal'][pipeline]), 
                        np.mean(dict_len_worst[dataset_name]['normal'][pipeline]),
                        xerr=np.std(dict_cov_worst[dataset_name]['normal'][pipeline])/np.sqrt(n_rep),
                        yerr=np.std(dict_len_worst[dataset_name]['normal'][pipeline])/np.sqrt(n_rep), 
                        color=dict_colors[pipeline], alpha=0.3)
            ax.scatter(np.mean(dict_cov_best[dataset_name]['normal'][pipeline]),
                       np.mean(dict_len_best[dataset_name]['normal'][pipeline]), 
                       marker=dict_markers['best'], color=dict_colors[pipeline],s=marker_size,alpha=alpha_data)
            ax.errorbar(np.mean(dict_cov_best[dataset_name]['normal'][pipeline]), 
                        np.mean(dict_len_best[dataset_name]['normal'][pipeline]),
                        xerr=np.std(dict_cov_best[dataset_name]['normal'][pipeline])/np.sqrt(n_rep),
                        yerr=np.std(dict_len_best[dataset_name]['normal'][pipeline])/np.sqrt(n_rep), 
                        color=dict_colors[pipeline], alpha=0.3)


        if small:
            ax.scatter(np.mean(dict_cov[dataset_name]['small'][pipeline]),
                       np.mean(dict_len[dataset_name]['small'][pipeline]), 
                       marker=dict_markers['iid'], color=dict_colors[pipeline],s=marker_size, facecolors='none',alpha=alpha_data)
            ax.errorbar(np.mean(dict_cov[dataset_name]['small'][pipeline]), 
                        np.mean(dict_len[dataset_name]['small'][pipeline]),
                        xerr=np.std(dict_cov[dataset_name]['small'][pipeline])/np.sqrt(n_rep),
                        yerr=np.std(dict_len[dataset_name]['small'][pipeline])/np.sqrt(n_rep), 
                        color=dict_colors[pipeline], alpha=0.3)
            ax.scatter(np.mean(dict_cov_worst[dataset_name]['small'][pipeline]),
                       np.mean(dict_len_worst[dataset_name]['small'][pipeline]), 
                       marker=dict_markers['worst'], color=dict_colors[pipeline],s=marker_size, facecolors='none',alpha=alpha_data)
            ax.errorbar(np.mean(dict_cov_worst[dataset_name]['small'][pipeline]), 
                        np.mean(dict_len_worst[dataset_name]['small'][pipeline]),
                        xerr=np.std(dict_cov_worst[dataset_name]['small'][pipeline])/np.sqrt(n_rep),
                        yerr=np.std(dict_len_worst[dataset_name]['small'][pipeline])/np.sqrt(n_rep), 
                        color=dict_colors[pipeline], alpha=0.3)
            ax.scatter(np.mean(dict_cov_best[dataset_name]['small'][pipeline]),
                       np.mean(dict_len_best[dataset_name]['small'][pipeline]), 
                       marker=dict_markers['best'], color=dict_colors[pipeline],s=marker_size, facecolors='none',alpha=alpha_data)
            ax.errorbar(np.mean(dict_cov_best[dataset_name]['small'][pipeline]), 
                        np.mean(dict_len_best[dataset_name]['small'][pipeline]),
                        xerr=np.std(dict_cov_best[dataset_name]['small'][pipeline])/np.sqrt(n_rep),
                        yerr=np.std(dict_len_best[dataset_name]['small'][pipeline])/np.sqrt(n_rep), 
                        color=dict_colors[pipeline], alpha=0.3)


for ax in [ax1,ax2,ax3,ax4]:
    ax.axvline(x=1-alpha, color='black', ls=':')
    ax.set_xlabel("Average coverage")

    
ax1.set_ylabel("Average length")


# Methods legend

handles = []
names = list( map(dict_methods.get, name_pipelines_to_plot) )
for idc,color in enumerate(list( map(dict_colors.get, name_pipelines_to_plot) )):
    handles.append(mlines.Line2D([], [], color=color, marker='o', linestyle='None', markersize=8))

if mask == 'Yes':
    fig.legend(handles, names, ncol=4, bbox_to_anchor=(0.63,0.13),handletextpad=0.1, 
               labelspacing=0.2, borderpad=0.3, handlelength=1.2, borderaxespad=1.1)
else:
    fig.legend(handles, names, ncol=4, bbox_to_anchor=(0.63,0.1),handletextpad=0.1, 
               labelspacing=0.2, borderpad=0.3, handlelength=1.2, borderaxespad=1.1)
    
# Coverage legend

handles = []
labels = []
for cov in list(dict_coverages.keys()):
    handles.append(mlines.Line2D([], [], color='black', marker=dict_markers[cov], linestyle='None', markersize=8))
    labels.append(dict_coverages[cov])

if mask == 'Yes':
    fig.legend(handles, labels, bbox_to_anchor=(0.95, 0.13),ncol=3,handletextpad=0.1, 
               labelspacing=0.2, borderpad=0.3, handlelength=1.2, borderaxespad=1.1)
else:
    fig.legend(handles, labels, bbox_to_anchor=(0.95, 0.1),ncol=3,handletextpad=0.1, 
               labelspacing=0.2, borderpad=0.3, handlelength=1.2, borderaxespad=1.1)


fig.tight_layout()

name_plot = 'plots/semi_synth'

if mask == 'Yes':
    plt.savefig(name_plot+'.pdf',bbox_inches='tight', dpi=300)
else:
    plt.savefig(name_plot+'_nomask.pdf',bbox_inches='tight', dpi=300)

plt.show()